<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/tam1444AH/UH-Insure-NSA.git"
REPO="UH-Insure-NSA"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

# save test.

Cloning repo 'UH-Insure-NSA'...
Cloning into 'UH-Insure-NSA'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 103 (delta 45), reused 40 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 58.67 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (45/45), done.
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 36.4 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [2]:
from model.test import test
test()

Hello World!


In [3]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

Logged in as: tam2003


In [4]:
from google.colab import auth
auth.authenticate_user()

# project_id = 'code-llm-finetuning'
# !gcloud config set project {project_id}

# GCS_PATH = "gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl"
LOCAL_PATH = "/tmp/source_code.jsonl"

# !gsutil ls {GCS_PATH}
# !gsutil -m cp {GCS_PATH} {LOCAL_PATH}

# !ls -lh {LOCAL_PATH}
# !head -n 2 {LOCAL_PATH}

import json

examples = [
    {
        "filename": "cryptol/examples/xor.cry",
        "content": """// xor.cry
property xor_inverse (a:[8], b:[8]) =
    ((a ^^ b) ^^ b) == a
"""
    },
    {
        "filename": "saw/examples/xor.saw",
        "content": """# xor.saw
prove xor_inverse with z3;
"""
    }
]

with open("/tmp/source_code.jsonl", "w") as f:
    for ex in examples:
        f.write(json.dumps(ex) + "\n")

print("Wrote dataset")
!head -n 5 /tmp/source_code.jsonl



#!gsutil cp gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl /tmp/source_code.jsonl
#!cat /tmp/source_code.jsonl

Wrote dataset
{"filename": "cryptol/examples/xor.cry", "content": "// xor.cry\nproperty xor_inverse (a:[8], b:[8]) =\n    ((a ^^ b) ^^ b) == a\n"}
{"filename": "saw/examples/xor.saw", "content": "# xor.saw\nprove xor_inverse with z3;\n"}


In [5]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen2.5-Coder-7B"  # Model checkpoint on the Hugging Face Hub
DATASET = "/tmp/source_code.jsonl"
DATA_COLUMN = "content"  # Column name containing the code content

SEQ_LENGTH = 2048  # Sequence length

# Training arguments
MAX_STEPS = 1#2000  # max_steps
BATCH_SIZE = 1  # batch_size
GR_ACC_STEPS = 16  # gradient_accumulation_steps
LR = 5e-4  # learning_rate
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.01  # weight_decay
NUM_WARMUP_STEPS = 30  # num_warmup_steps
EVAL_FREQ = 100  # eval_freq
SAVE_FREQ = 100  # save_freq
LOG_FREQ = 25  # log_freq
OUTPUT_DIR = "peft-FT-2.5-Coder-7b"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.5  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate

# LORA
LORA_R = 8  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = "c_proj,c_attn,q_attn,c_fc,c_proj"  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 0

In [6]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)

set_seed(SEED)

In [7]:
from datasets import load_dataset

DATASET = LOCAL_PATH
DATA_COLUMN = "content"

dataset = load_dataset("json", data_files={"train": DATASET})
train_data = dataset["train"]

# quick sanity check: make sure the field exists
print(train_data[0].keys())
assert DATA_COLUMN in train_data.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"


Generating train split: 0 examples [00:00, ? examples/s]

dict_keys(['filename', 'content'])


In [8]:
import torch
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

def chars_token_ratio(dataset, tokenizer, data_column, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """

    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        total_characters += len(example[data_column])
        total_tokens += len(tokenizer(example[data_column]).tokens())

    return total_characters / total_tokens


chars_per_token = chars_token_ratio(train_data, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

  0%|          | 2/400 [00:00<00:00, 818.32it/s]

The character to token ratio of the dataset is: 2.64


In [9]:
from model.dataset.util import *

In [10]:
from model.dataset.constantLengthDataset import ConstantLengthDataset


train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        infinite=True,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)
eval_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        #valid_data,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)

In [11]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

load_in_8bit = False

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

device_map = "auto"#{"": 0}

model = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config,
        device_map=device_map,
        use_cache=False,  # We will be using gradient checkpointing
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
)

model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

In [12]:
blk = model.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

ATTN: Qwen2Attention(
  (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
  (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
  (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
  (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
)
MLP: Qwen2MLP(
  (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
  (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
  (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
  (act_fn): SiLU()
)


In [13]:
# Set up lora
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


In [14]:
train_data.start_iteration = 0


training_args = TrainingArguments(
    output_dir=f"j05hr3d/{OUTPUT_DIR}",
    dataloader_drop_last=True,
    eval_strategy="steps",
    save_strategy="steps",
    max_steps=MAX_STEPS,
    eval_steps=EVAL_FREQ,
    save_steps=SAVE_FREQ,
    logging_steps=LOG_FREQ,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=NUM_WARMUP_STEPS,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=True,
    fp16=FP16,
    bf16=BF16,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=True,
    include_tokens_per_second=True,
    report_to=[]
)

In [15]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

print("Training...")
trainer.train()

Training...


Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.


Step,Training Loss,Validation Loss


TrainOutput(global_step=1, training_loss=1.8659778833389282, metrics={'train_runtime': 25.8798, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.039, 'train_tokens_per_second': 1266.161, 'total_flos': 1394108836872192.0, 'train_loss': 1.8659778833389282, 'epoch': 1.0})

In [16]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t-FT-2.5-Coder-7b/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ...-Coder-7b/adapter_model.safetensors:  52%|#####1    | 41.8MB / 80.8MB            

CommitInfo(commit_url='https://huggingface.co/tam2003/peft-FT-2.5-Coder-7b/commit/a82a4d9a4c6a5e0a3e6356b032654333a303d762', commit_message='End of training', commit_description='', oid='a82a4d9a4c6a5e0a3e6356b032654333a303d762', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tam2003/peft-FT-2.5-Coder-7b', endpoint='https://huggingface.co', repo_type='model', repo_id='tam2003/peft-FT-2.5-Coder-7b'), pr_revision=None, pr_num=None)